In [1]:

import sys
sys.path.append('/home/techt/Desktop/a4s')


### Step 1: Initial Data Loading

In [2]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Arrhythmia/GSE41177'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Region-specific gene expression profiles in left atria of patients with valvular atrial fibrillation"
!Series_summary	"Of 54,675 expressed sequence tags, microarray analysis revealed that 391 genes were differently expressed (>1.5-fold difference) between LA-PV junction and LAA, including genes related to arrhythmia, cell death, fibrosis, hypertrophy, and inflammation. Microarray and q-PCR produced parallel results in analyzing the expression of particular genes. The expression of paired like homeodomain-2 (PITX2) and its target protein (short stature homeobox-2 [SHOX2]) was greater in LA-PV junction than in LAA, which may contribute to arrhythmogenesis. Five genes related to thrombogenesis were up-regulated in LAA, which may implicate for the preferential thrombus formation in LAA. Genes related to fibrosis were highly expressed in LAA, which was reflected by intense ultrastructural changes in this region"
!Series_overall_design	"Paired LA-PV jun

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
is_gene_available = False
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_gender = None  # define the functions when applicable

# Check if the dataset contains gene expression data
is_gene_available = True  # Given the dataset is a microarray analysis

# Check and assign keys for available variables
sample_characteristics_dict = {
    0: ['organ: left atrial appendage', 'organ: left atrial junction'], 
    1: ['gender: female', 'gender: male'], 
    2: ['age: 62Y', 'age: 43Y', 'age: 55Y', 'age: 65Y', 'age: 61Y', 'age: 64Y', 'age: 47Y', 'age: 60Y', 'age: 71Y', 'age: 32Y', 'age: 59Y', 'age: 56Y', 'age: 51Y', 'age: 66Y', 'age: 36Y'], 
    3: ['af duration: 0M', 'af duration: 10M', 'af duration: 110M', 'af duration: 15M', 'af duration: >1M', 'af duration: 72M', 'af duration: 102M', 'af duration: 48M', 'af duration: 100M', 'af duration: 73M', 'af duration: 14M', 'af duration: 150M', 'af duration: 78M']
}

gender_row = 1  # Gender data is available in the dictionary under key 1
age_row = 2  # Age data is available in the dictionary under key 2

def convert_trait(value):
    return None  # Trait 'Arrhythmia' data is not clearly defined in the dataset

def convert_age(value):
    # Ensure we only extract the numerical age part and convert to integer
    try:
        age_value = int(value.split(": ")[1][:-1])
        return age_value
    except (IndexError, ValueError):
        return None

def convert_gender(value):
    # Convert gender to binary with male: 1 and female: 0
    gender = value.split(": ")[1].strip().lower()
    if gender == 'male':
        return 1
    elif gender == 'female':
        return 0
    else:
        return None

save_cohort_info('GSE41177', './preprocessed/Arrhythmia/cohort_info.json', is_gene_available, trait_row is not None)

# Assuming clinical_data has been previously defined
if trait_row is not None:
    selected_clinical_data = geo_select_clinical_features(clinical_data, 'Arrhythmia', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
    csv_path = './preprocessed/Arrhythmia/trait_data/GSE41177.csv'
    selected_clinical_data.to_csv(csv_path)
    print(preview_df(selected_clinical_data))
